In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from PIL import Image
import pandas as pd
from utils import image_grid
import inflect
from nltk.corpus import words
import zipfile
import os
from pathlib import Path
import numpy as np
import itertools
os.environ['OPENAI_API_KEY'] = 'sk-proj-gECJbZO1y4sk3NoUB1SbT3BlbkFJ5QdYttqNjmS1hSsGQIJY'
from openai import OpenAI

## retrieve

In [3]:
client = OpenAI()
client.batches.retrieve('batch_YXN08eUOrrXROdAUhdmuCP8M')

Batch(id='batch_YXN08eUOrrXROdAUhdmuCP8M', completion_window='24h', created_at=1723525608, endpoint='/v1/chat/completions', input_file_id='file-f1RzRtzL0KpZjf0XEdai0dsl', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1723525617, error_file_id=None, errors=None, expired_at=None, expires_at=1723612008, failed_at=None, finalizing_at=1723525616, in_progress_at=1723525609, metadata={'description': 'new_selected_dataset_all'}, output_file_id='file-ieiMvL9AHZwAtpTvwppiJxGp', request_counts=BatchRequestCounts(completed=30, failed=0, total=30))

In [4]:
output_file_id = client.batches.retrieve('batch_YXN08eUOrrXROdAUhdmuCP8M').output_file_id
content = client.files.content(output_file_id)

output_file_path = "new_dataset/output/test_0_30_add.jsonl"

with open(output_file_path, 'w') as f:
    f.write(content.content.decode("utf-8"))

## process

In [16]:
def extract_data(outputs):
    df = []
    for output in outputs:
        df.append({
            "id": output["id"],
            "custom_id": output["custom_id"],
            "status_code": output["response"]["status_code"],
            "model": output["response"]["body"]["model"],
            "prompt_tokens": output["response"]["body"]["usage"]["prompt_tokens"],
            "completion_tokens": output["response"]["body"]["usage"]["completion_tokens"],
            "answer": output["response"]["body"]["choices"][0]["message"]["content"],
        })

    return pd.DataFrame(df)

In [17]:
paths = [
    # "new_dataset/output/test_0_30_select.jsonl",
    # "new_dataset/output/test_0_30_add.jsonl",
    "new_dataset/output/test_0_30_add_ver2.jsonl",
]
outputs = []
for path in paths:
    outputs += ([json.loads(i) for i in open(path).readlines()])

df = extract_data(outputs)
df.head(2)

,id,custom_id,status_code,model,prompt_tokens,completion_tokens,answer
0,batch_req_MD1a7KMpzBVGHrZrHJgLRwUb,0,200,gpt-4o-2024-05-13,887,10,"birds, airplane, fireworks, kite, moon."
1,batch_req_S7ppgbwLxWGmCJnxXi2WMlsZ,1,200,gpt-4o-2024-05-13,887,12,"Butterfly, sunglasses, bird, small flower, but..."


In [18]:
def parse_answer(text):
    parsed_text = text.lower().split(', ')
    if parsed_text[-1].endswith('.'):
        parsed_text[-1] = parsed_text[-1][:-1]

    return parsed_text

name_mapping = lambda x: f'answer_{x % 5}'
parsed_columns = df['answer'].apply(parse_answer)
parsed_df = pd.DataFrame(parsed_columns.tolist(), columns=[name_mapping(i) for i in range(5)])
for col in parsed_df.columns:
    df[col] = parsed_df[col]

input_df = pd.read_csv("new_selected_dataset_all.csv")
input_df["custom_id"] = input_df.index
df = input_df.join(df, on="custom_id", how="inner", lsuffix="_left", rsuffix="_right")
columns_to_drop = [col for col in df.columns if col.endswith('_left')]
df.drop(columns=columns_to_drop, inplace=True)
df.columns = [col.replace('_right', '') for col in df.columns]

# save_path = "new_dataset/filled_dataframe/test_0_30_add.csv"
save_path = "new_dataset/filled_dataframe/test_0_30_add_ver2.csv"
os.makedirs(Path(save_path).parent, exist_ok=True)
df.to_csv(save_path, index=False)

df.head(2)

,custom_id,img_path,source,id,custom_id,status_code,model,prompt_tokens,completion_tokens,answer,answer_0,answer_1,answer_2,answer_3,answer_4
0,0,data_original/DIV2K_valid_HR/0814.png,DIV2K_valid_HR,batch_req_MD1a7KMpzBVGHrZrHJgLRwUb,0,200,gpt-4o-2024-05-13,887,10,"birds, airplane, fireworks, kite, moon.",birds,airplane,fireworks,kite,moon
1,1,data_original/DIV2K_valid_HR/0809.png,DIV2K_valid_HR,batch_req_S7ppgbwLxWGmCJnxXi2WMlsZ,1,200,gpt-4o-2024-05-13,887,12,"Butterfly, sunglasses, bird, small flower, but...",butterfly,sunglasses,bird,small flower,butterfly


In [30]:
from utils import zip_files

save_path = "new_dataset/inspect/test_0_30.zip"
os.makedirs(Path(save_path).parent, exist_ok=True)
zip_files(df["img_path"].to_list(), save_path)

DIV2K_valid_HR/0814.jpg
DIV2K_valid_HR/0809.jpg
Flickr2K/000031.jpg
Flickr2K/000048.jpg
Flickr2K/000149.jpg
Flickr2K/000293.jpg
Flickr2K/000298.jpg
Flickr2K/000617.jpg
Flickr2K/001139.jpg
Flickr2K/001334.jpg
Flickr2K/001366.jpg
Flickr2K/001404.jpg
Flickr2K/001583.jpg
Flickr2K/001617.jpg
Flickr2K/001683.jpg
Flickr2K/001843.jpg
Flickr2K/001857.jpg
Flickr2K/001893.jpg
Flickr2K/002038.jpg
Flickr2K/002097.jpg
Flickr2K/002243.jpg
Flickr2K/002259.jpg
Flickr2K/002277.jpg
Flickr2K/002282.jpg
Flickr2K/002302.jpg
Flickr2K/002305.jpg
Flickr2K/002544.jpg
Flickr1024_val/0028_L.jpg
HRWSI_val/65878_z98_GeneveBuste.jpg
DIV2K_valid_HR/0806.jpg
Created ZIP file: new_dataset/inspect/test_0_30.zip
